## Shapelets

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from keras.optimizers import Adagrad
from tslearn.shapelets import ShapeletModel
from tslearn.shapelets import grabocka_params_to_shapelet_size_dict
from tslearn.preprocessing import TimeSeriesScalerMinMax

Using TensorFlow backend.
/Users/gianmarcofranchini/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/gianmarcofranchini/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/gianmarcofranchini/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/User

In [3]:
df = pd.read_csv('../occupancy_data/time_series/original/training_ts.csv', parse_dates = True, index_col=['date']).drop('Unnamed: 0', 1)

Il problema è il seguente: nell'identificazione e nell'uso delle shapelet, si dà per scontato che tutta la time series sia associata a una sola classe. Questo non è ovviamente il caso delle time series costruite sul dataset che abbiamo noi: ogni timestamp (e quindi ogni record) è collegato a una classe propria. Al limite, si può forzare la mano dividendo la time series in time series le più grandi possibili in cui ciascun record sia collegato alla stessa classe.

In [4]:
def breakdown_ts(ts, attribute):
    empty = {}
    occ = {}
    counter_empty = 1
    counter_occ = 1
    iterations = 0
    for index in ts.index:
        iterations += 1
        class_label = ts.at[index, 'Occupancy']
        if class_label == 0:
            if counter_empty in empty.keys():
                empty[counter_empty].append((index, ts.at[index, attribute], class_label))
            else:
                empty[counter_empty] = [(index, ts.at[index, attribute], class_label)]
                if iterations != 1:
                    counter_occ += 1
        else:
            if counter_occ in occ.keys():
                occ[counter_occ].append((index, ts.at[index, attribute], class_label))
            else:
                occ[counter_occ] = [(index, ts.at[index, attribute], class_label)]
                if iterations != 1:
                    counter_empty += 1
    return empty, occ

### Temperature

In [13]:
ts1 = df.drop(['Humidity','Light','CO2','HumidityRatio'], 1).resample('10T').median().dropna()

In [14]:
empty, occ = breakdown_ts(ts1, 'Temperature')

In [15]:
empty

{1: [(Timestamp('2015-02-04 18:10:00'), 22.89, 0.0),
  (Timestamp('2015-02-04 18:20:00'), 22.6666666666667, 0.0),
  (Timestamp('2015-02-04 18:30:00'), 22.5, 0.0),
  (Timestamp('2015-02-04 18:40:00'), 22.39, 0.0),
  (Timestamp('2015-02-04 18:50:00'), 22.29, 0.0),
  (Timestamp('2015-02-04 19:00:00'), 22.245, 0.0),
  (Timestamp('2015-02-04 19:10:00'), 22.2, 0.0),
  (Timestamp('2015-02-04 19:20:00'), 22.1, 0.0),
  (Timestamp('2015-02-04 19:30:00'), 22.1, 0.0),
  (Timestamp('2015-02-04 19:40:00'), 21.93583333333335, 0.0),
  (Timestamp('2015-02-04 19:50:00'), 21.89, 0.0),
  (Timestamp('2015-02-04 20:00:00'), 21.79, 0.0),
  (Timestamp('2015-02-04 20:10:00'), 21.7, 0.0),
  (Timestamp('2015-02-04 20:20:00'), 21.7, 0.0),
  (Timestamp('2015-02-04 20:30:00'), 21.7, 0.0),
  (Timestamp('2015-02-04 20:40:00'), 21.6, 0.0),
  (Timestamp('2015-02-04 20:50:00'), 21.6, 0.0),
  (Timestamp('2015-02-04 21:00:00'), 21.5, 0.0),
  (Timestamp('2015-02-04 21:10:00'), 21.5, 0.0),
  (Timestamp('2015-02-04 21:20:00'

Now comes another tricky part: we need all time series to be the same length (because of the implementation of the shapelet method we're using). This means we need to discard time series which are too short and trim all time series down to the same length, possibly subdividing a long time series in shorter time series (i.e. if our chosen length is 20 and we have a time series of length 67, we'll extract three time series of length 20 from it and trim the last 7 records).

In [16]:
list_arrays_empty = []
for key in empty.keys():
    series = []
    for record in empty[key]:
        series.append(record[1])
    list_arrays_empty.append(np.array(series))

In [17]:
temp = []
for array in list_arrays_empty:
    # I've chosen 20 because it was the threshold that made more sense given the length of the time series 
    # associated with 'Occupancy' = 1 (to do this, I first checked said lengths)
    if len(array) >= 20:
        temp.append(array)
list_arrays_empty = temp

In [18]:
len(list_arrays_empty)

7

In [19]:
for array in list_arrays_empty:
    print(len(array))

81
82
376
87
85
92
131


In [20]:
list_arrays_occ = []
for key in occ.keys():
    series = []
    for record in occ[key]:
        series.append(record[1])
    list_arrays_occ.append(np.array(series))

In [21]:
temp = []
for array in list_arrays_occ:
    if len(array) >= 20:
        temp.append(array)
list_arrays_occ = temp

In [22]:
len(list_arrays_occ)

9

In [23]:
for array in list_arrays_occ:
    print(len(array))

29
24
30
26
26
28
28
24
26


Now we need to apply the division and trimming process described above to the two lists: list_arrays_occ and \_empty

In [24]:
temp = []
for array in list_arrays_occ:
    if len(array)!= 20:
        counter = 0
        temp2 = []
        for item in array:
            temp2.append(item)
            counter +=1
            if counter == 20:
                temp.append(temp2)
                temp2 = []
                counter = 0
list_arrays_occ = temp

In [25]:
temp = []
for array in list_arrays_empty:
    if len(array)!= 20:
        counter = 0
        temp2 = []
        for item in array:
            temp2.append(item)
            counter +=1
            if counter == 20:
                temp.append(temp2)
                temp2 = []
                counter = 0
list_arrays_empty = temp

In [26]:
len(list_arrays_occ)

9

In [27]:
len(list_arrays_empty)

44

In [28]:
list_arrays_occ

[[20.74875,
  20.945,
  21.025,
  21.1,
  21.29,
  21.323333333333302,
  21.365,
  21.403750000000002,
  21.5375,
  21.89,
  22.05,
  22.1,
  22.1,
  22.1,
  22.1,
  22.1,
  22.05,
  22.1,
  22.054166666666653,
  22.0666666666667],
 [22.2,
  22.2,
  22.2225,
  22.29,
  22.39,
  22.525,
  22.6,
  22.55,
  22.5,
  22.5,
  22.5,
  22.4175,
  22.39,
  22.408333333333346,
  22.39,
  22.360833333333346,
  22.29,
  22.2675,
  22.29,
  22.245],
 [20.2675,
  20.5666666666667,
  20.79,
  20.89,
  20.856666666666698,
  20.865,
  20.865,
  20.89,
  20.958750000000002,
  21.05,
  21.1,
  21.2675,
  21.315,
  21.34,
  21.39,
  21.34,
  21.315,
  21.625,
  21.73,
  21.7225],
 [21.7,
  21.79,
  21.89,
  22.0,
  22.1,
  22.2,
  22.2,
  22.2,
  22.2,
  22.1,
  22.075,
  22.1,
  22.0,
  22.0,
  21.89,
  21.89,
  21.745,
  21.77875,
  21.76,
  21.79],
 [19.6333333333333,
  19.814999999999998,
  20.075000000000003,
  20.23,
  20.39,
  20.5,
  20.6,
  20.7,
  20.79,
  20.79,
  20.89,
  21.0,
  21.0666666666

In [29]:
time_series = []
for array in list_arrays_occ:
    time_series.append(np.array(array))
for array in list_arrays_empty:
    time_series.append(np.array(array))
time_series = np.array(time_series)

In [30]:
time_series

array([[20.74875   , 20.945     , 21.025     , ..., 22.1       ,
        22.05416667, 22.06666667],
       [22.2       , 22.2       , 22.2225    , ..., 22.2675    ,
        22.29      , 22.245     ],
       [20.2675    , 20.56666667, 20.79      , ..., 21.625     ,
        21.73      , 21.7225    ],
       ...,
       [19.66666667, 19.6       , 19.6       , ..., 19.575     ,
        19.6       , 19.6       ],
       [19.6       , 19.6       , 19.63333333, ..., 20.0875    ,
        20.1875    , 20.29      ],
       [20.56666667, 20.79      , 20.79      , ..., 20.52916667,
        20.5       , 20.29      ]])

In [31]:
time_series.shape

(53, 20)

In [32]:
len(time_series)

53

In [33]:
classes = np.array([1 for i in range(0,9)] + [0 for i in range(0,44)])

In [34]:
sizes = grabocka_params_to_shapelet_size_dict(n_ts=9+44,
                                              ts_sz=20,
                                              n_classes=2,
                                              l=0.1,
                                              r=1)

In [35]:
shp_finder = ShapeletModel(n_shapelets_per_size=sizes,
                        optimizer="sgd",
                        weight_regularizer=.01,
                        max_iter=200,
                        verbose=1)

In [36]:
shp_finder.fit(time_series,classes)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/200
53/53 [==============================] - 2s 31ms/step - loss: 0.3519 - binary_accuracy: 0.8113 - binary_crossentropy: 0.3431
Epoch 2/200
53/53 [==============================] - 0s 136us/step - loss: 0.3512 - binary_accuracy: 0.8113 - binary_crossentropy: 0.3424
Epoch 3/200
53/53 [==============================] - 0s 136us/step - loss: 0.3505 - binary_accuracy: 0.8113 - binary_crossentropy: 0.3417
Epoch 4/200
53/53 [==============================] - 0s 127us/step - loss: 0.3499 - binary_accuracy: 0.8113 - binary_crossentropy: 0.3410
Epoch 5/200
53/53 [==============================] - 0s 129us/step - loss: 0.3492 - binary_accuracy: 0.8113 - binary_crossentropy: 0.3404
Epoch 6/200
53/53 [==============================] - 0s 148us/step - loss: 0.3486 - binary_accuracy: 0.8113 - binary_crossentropy: 0.3397
Epoch 7/200
53/53 [==============================] - 0s 119us/step - loss: 0.3

Epoch 56/200
53/53 [==============================] - 0s 115us/step - loss: 0.3242 - binary_accuracy: 0.8679 - binary_crossentropy: 0.3145
Epoch 57/200
53/53 [==============================] - 0s 154us/step - loss: 0.3238 - binary_accuracy: 0.8679 - binary_crossentropy: 0.3141
Epoch 58/200
53/53 [==============================] - 0s 121us/step - loss: 0.3234 - binary_accuracy: 0.8679 - binary_crossentropy: 0.3138
Epoch 59/200
53/53 [==============================] - 0s 143us/step - loss: 0.3231 - binary_accuracy: 0.8679 - binary_crossentropy: 0.3134
Epoch 60/200
53/53 [==============================] - 0s 119us/step - loss: 0.3227 - binary_accuracy: 0.8679 - binary_crossentropy: 0.3130
Epoch 61/200
53/53 [==============================] - 0s 144us/step - loss: 0.3223 - binary_accuracy: 0.8679 - binary_crossentropy: 0.3126
Epoch 62/200
53/53 [==============================] - 0s 249us/step - loss: 0.3220 - binary_accuracy: 0.8679 - binary_crossentropy: 0.3123
Epoch 63/200
53/53 [=======

Epoch 115/200
53/53 [==============================] - 0s 125us/step - loss: 0.3071 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2966
Epoch 116/200
53/53 [==============================] - 0s 110us/step - loss: 0.3068 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2964
Epoch 117/200
53/53 [==============================] - 0s 143us/step - loss: 0.3066 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2961
Epoch 118/200
53/53 [==============================] - 0s 124us/step - loss: 0.3064 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2959
Epoch 119/200
53/53 [==============================] - 0s 127us/step - loss: 0.3062 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2957
Epoch 120/200
53/53 [==============================] - 0s 192us/step - loss: 0.3060 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2954
Epoch 121/200
53/53 [==============================] - 0s 130us/step - loss: 0.3057 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2952
Epoch 122/200
53/53 

Epoch 174/200
53/53 [==============================] - 0s 127us/step - loss: 0.2961 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2849
Epoch 175/200
53/53 [==============================] - 0s 113us/step - loss: 0.2959 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2848
Epoch 176/200
53/53 [==============================] - 0s 126us/step - loss: 0.2958 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2846
Epoch 177/200
53/53 [==============================] - 0s 131us/step - loss: 0.2956 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2845
Epoch 178/200
53/53 [==============================] - 0s 112us/step - loss: 0.2955 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2843
Epoch 179/200
53/53 [==============================] - 0s 131us/step - loss: 0.2953 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2841
Epoch 180/200
53/53 [==============================] - 0s 185us/step - loss: 0.2952 - binary_accuracy: 0.8679 - binary_crossentropy: 0.2840
Epoch 181/200
53/53 

ShapeletModel(batch_size=256, max_iter=200, n_shapelets_per_size={2: 3},
              optimizer='sgd', random_state=None, shapelet_length=0.15,
              total_lengths=3, verbose=1, verbose_level=None,
              weight_regularizer=0.01)

Ecco le prime shapelet che possiamo plottare (sono un po' piccole, però):

In [49]:
shp_finder.shapelets_

array([array([[19.69297218],
       [19.69006348]]),
       array([[20.79735374],
       [20.80137062]]),
       array([[22.39876938],
       [22.40593529]])], dtype=object)

In [50]:
shp_finder = ShapeletModel(n_shapelets_per_size={5:3},
                        optimizer="sgd",
                        weight_regularizer=.01,
                        max_iter=200,
                        verbose=1)

In [51]:
shp_finder.fit(time_series,classes)

Epoch 1/200
53/53 [==============================] - 1s 17ms/step - loss: 2.0032 - binary_accuracy: 0.2264 - binary_crossentropy: 1.9847
Epoch 2/200
53/53 [==============================] - 0s 106us/step - loss: 1.9306 - binary_accuracy: 0.2264 - binary_crossentropy: 1.9124
Epoch 3/200
53/53 [==============================] - 0s 138us/step - loss: 1.8596 - binary_accuracy: 0.2264 - binary_crossentropy: 1.8417
Epoch 4/200
53/53 [==============================] - 0s 128us/step - loss: 1.7903 - binary_accuracy: 0.2264 - binary_crossentropy: 1.7726
Epoch 5/200
53/53 [==============================] - 0s 310us/step - loss: 1.7227 - binary_accuracy: 0.2453 - binary_crossentropy: 1.7053
Epoch 6/200
53/53 [==============================] - 0s 114us/step - loss: 1.6567 - binary_accuracy: 0.2830 - binary_crossentropy: 1.6396
Epoch 7/200
53/53 [==============================] - 0s 130us/step - loss: 1.5925 - binary_accuracy: 0.2830 - binary_crossentropy: 1.5756
Epoch 8/200
53/53 [================

53/53 [==============================] - 0s 138us/step - loss: 0.4175 - binary_accuracy: 0.8302 - binary_crossentropy: 0.4025
Epoch 61/200
53/53 [==============================] - 0s 148us/step - loss: 0.4146 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3996
Epoch 62/200
53/53 [==============================] - 0s 149us/step - loss: 0.4117 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3968
Epoch 63/200
53/53 [==============================] - 0s 147us/step - loss: 0.4090 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3941
Epoch 64/200
53/53 [==============================] - 0s 117us/step - loss: 0.4064 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3914
Epoch 65/200
53/53 [==============================] - 0s 156us/step - loss: 0.4039 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3889
Epoch 66/200
53/53 [==============================] - 0s 167us/step - loss: 0.4015 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3865
Epoch 67/200
53/53 [====================

Epoch 119/200
53/53 [==============================] - 0s 146us/step - loss: 0.3356 - binary_accuracy: 0.8113 - binary_crossentropy: 0.3198
Epoch 120/200
53/53 [==============================] - 0s 151us/step - loss: 0.3350 - binary_accuracy: 0.8113 - binary_crossentropy: 0.3191
Epoch 121/200
53/53 [==============================] - 0s 149us/step - loss: 0.3343 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3185
Epoch 122/200
53/53 [==============================] - 0s 147us/step - loss: 0.3337 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3178
Epoch 123/200
53/53 [==============================] - 0s 134us/step - loss: 0.3331 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3172
Epoch 124/200
53/53 [==============================] - 0s 148us/step - loss: 0.3325 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3165
Epoch 125/200
53/53 [==============================] - 0s 151us/step - loss: 0.3319 - binary_accuracy: 0.8302 - binary_crossentropy: 0.3159
Epoch 126/200
53/53 

Epoch 178/200
53/53 [==============================] - 0s 118us/step - loss: 0.3103 - binary_accuracy: 0.8302 - binary_crossentropy: 0.2937
Epoch 179/200
53/53 [==============================] - 0s 107us/step - loss: 0.3101 - binary_accuracy: 0.8302 - binary_crossentropy: 0.2934
Epoch 180/200
53/53 [==============================] - 0s 117us/step - loss: 0.3098 - binary_accuracy: 0.8491 - binary_crossentropy: 0.2931
Epoch 181/200
53/53 [==============================] - 0s 135us/step - loss: 0.3095 - binary_accuracy: 0.8491 - binary_crossentropy: 0.2929
Epoch 182/200
53/53 [==============================] - 0s 141us/step - loss: 0.3092 - binary_accuracy: 0.8491 - binary_crossentropy: 0.2926
Epoch 183/200
53/53 [==============================] - 0s 140us/step - loss: 0.3090 - binary_accuracy: 0.8491 - binary_crossentropy: 0.2923
Epoch 184/200
53/53 [==============================] - 0s 139us/step - loss: 0.3087 - binary_accuracy: 0.8491 - binary_crossentropy: 0.2920
Epoch 185/200
53/53 

ShapeletModel(batch_size=256, max_iter=200, n_shapelets_per_size={5: 3},
              optimizer='sgd', random_state=None, shapelet_length=0.15,
              total_lengths=3, verbose=1, verbose_level=None,
              weight_regularizer=0.01)

In [56]:
shapelets = shp_finder.shapelets_

Ed ecco le successive (ho costretto il classificatore a trovarne di più lunghe):

In [62]:
shapelets

array([array([[19.69922447],
       [19.69902611],
       [19.70206642],
       [19.70714951],
       [19.7134819 ]]),
       array([[20.71943474],
       [20.70772362],
       [20.69440842],
       [20.67897606],
       [20.66368484]]),
       array([[22.31327629],
       [22.32262039],
       [22.32759285],
       [22.32819939],
       [22.31933403]])], dtype=object)

In pratica, ogni array corrisponde alle coordinate di Temperature di una shapelet. Per plottarle, bisogna aggiungere un'altra coordinata, che va sulle x e sono le ore: a ogni coordinata di Temperature corrisponde un'ora sull'asse x (non dev'essere un'ora specifica, basta che ci sia scritto "1", per quella successiva "2" etc.

### Light

In [5]:
ts2 = df.drop(['Humidity','Temperature','CO2','HumidityRatio'], 1).resample('10T').median().dropna()

In [6]:
empty, occ = breakdown_ts(ts2, 'Light')

In [7]:
empty

{1: [(Timestamp('2015-02-04 18:10:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 18:20:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 18:30:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 18:40:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 18:50:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 19:00:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 19:10:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 19:20:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 19:30:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 19:40:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 19:50:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 20:00:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 20:10:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 20:20:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 20:30:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 20:40:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 20:50:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 21:00:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 21:10:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 21:20:00'), 0.0, 0.0),
  (Timestamp('2015-02-04 21:30:00'), 

Now comes another tricky part: we need all time series to be the same length (because of the implementation of the shapelet method we're using). This means we need to discard time series which are too short and trim all time series down to the same length, possibly subdividing a long time series in shorter time series (i.e. if our chosen length is 20 and we have a time series of length 67, we'll extract three time series of length 20 from it and trim the last 7 records).

In [8]:
list_arrays_empty = []
for key in empty.keys():
    series = []
    for record in empty[key]:
        series.append(record[1])
    list_arrays_empty.append(np.array(series))

In [9]:
temp = []
for array in list_arrays_empty:
    # I've chosen 20 because it was the threshold that made more sense given the length of the time series 
    # associated with 'Occupancy' = 1 (to do this, I first checked said lengths)
    if len(array) >= 20:
        temp.append(array)
list_arrays_empty = temp

In [10]:
len(list_arrays_empty)

7

In [99]:
for array in list_arrays_empty:
    print(len(array))

81
82
376
87
85
92
131


In [11]:
list_arrays_occ = []
for key in occ.keys():
    series = []
    for record in occ[key]:
        series.append(record[1])
    list_arrays_occ.append(np.array(series))

In [12]:
temp = []
for array in list_arrays_occ:
    if len(array) >= 20:
        temp.append(array)
list_arrays_occ = temp

In [102]:
len(list_arrays_occ)

9

In [103]:
for array in list_arrays_occ:
    print(len(array))

29
24
30
26
26
28
28
24
26


Now we need to apply the division and trimming process described above to the two lists: list_arrays_occ and \_empty

In [13]:
temp = []
for array in list_arrays_occ:
    if len(array)!= 20:
        counter = 0
        temp2 = []
        for item in array:
            temp2.append(item)
            counter +=1
            if counter == 20:
                temp.append(temp2)
                temp2 = []
                counter = 0
list_arrays_occ = temp

In [14]:
temp = []
for array in list_arrays_empty:
    if len(array)!= 20:
        counter = 0
        temp2 = []
        for item in array:
            temp2.append(item)
            counter +=1
            if counter == 20:
                temp.append(temp2)
                temp2 = []
                counter = 0
list_arrays_empty = temp

In [106]:
len(list_arrays_occ)

9

In [107]:
len(list_arrays_empty)

44

In [15]:
list_arrays_occ

[[415.0,
  399.5,
  405.0,
  419.0,
  424.6,
  433.0,
  441.75,
  449.4166666666665,
  472.125,
  495.5,
  505.5,
  478.0,
  454.0,
  454.0,
  449.5,
  454.0,
  469.0,
  461.5,
  454.0,
  464.0],
 [444.0,
  438.0,
  441.0,
  494.0,
  532.625,
  523.0,
  515.75,
  454.75,
  460.0,
  474.0,
  474.0,
  452.333333333333,
  444.0,
  438.0,
  444.0,
  441.0,
  438.0,
  433.0,
  433.0,
  433.0],
 [411.0,
  411.0,
  419.0,
  426.0,
  433.0,
  433.0,
  433.0,
  433.0,
  441.6666666666665,
  440.25,
  449.5,
  456.0,
  456.0,
  455.5,
  454.75,
  454.75,
  471.5,
  499.625,
  482.75,
  479.0],
 [551.0,
  546.5,
  534.666666666667,
  557.25,
  556.75,
  534.25,
  523.5,
  518.0,
  509.0,
  499.0,
  487.333333333333,
  479.0,
  474.5,
  469.0,
  469.0,
  460.375,
  460.0,
  460.0,
  444.0,
  444.0],
 [395.3333333333335,
  390.75,
  405.0,
  419.0,
  433.0,
  441.0,
  438.0,
  438.0,
  438.0,
  439.625,
  439.0,
  454.0,
  454.0,
  458.0,
  479.0,
  489.0,
  475.25,
  474.0,
  464.0,
  464.0],
 [26

In [16]:
time_series = []
for array in list_arrays_occ:
    time_series.append(np.array(array))
for array in list_arrays_empty:
    time_series.append(np.array(array))
time_series = np.array(time_series)

In [17]:
time_series

array([[415.        , 399.5       , 405.        , ..., 461.5       ,
        454.        , 464.        ],
       [444.        , 438.        , 441.        , ..., 433.        ,
        433.        , 433.        ],
       [411.        , 411.        , 419.        , ..., 499.625     ,
        482.75      , 479.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,  37.        ,
         46.75      ,  73.        ],
       [213.33333333, 134.6       ,  53.        , ...,  60.125     ,
         37.        ,  31.        ]])

In [18]:
time_series.shape

(53, 20)

In [112]:
len(time_series)

53

In [19]:
classes = np.array([1 for i in range(0,9)] + [0 for i in range(0,44)])

In [20]:
sizes = grabocka_params_to_shapelet_size_dict(n_ts=9+44,
                                              ts_sz=20,
                                              n_classes=2,
                                              l=0.1,
                                              r=1)

In [21]:
shp_finder = ShapeletModel(n_shapelets_per_size=sizes,
                        optimizer="sgd",
                        weight_regularizer=.01,
                        max_iter=200,
                        verbose=1)

In [23]:
shp_finder.fit(time_series,classes)

Epoch 1/200
53/53 [==============================] - 1s 17ms/step - loss: 0.0124 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 2/200
53/53 [==============================] - 0s 128us/step - loss: 0.0124 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 3/200
53/53 [==============================] - 0s 118us/step - loss: 0.0124 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 4/200
53/53 [==============================] - 0s 138us/step - loss: 0.0124 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 5/200
53/53 [==============================] - 0s 154us/step - loss: 0.0124 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 6/200
53/53 [==============================] - 0s 160us/step - loss: 0.0124 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 7/200
53/53 [==============================] - 0s 153us/step - loss: 0.0124 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 8

Epoch 59/200
53/53 [==============================] - 0s 121us/step - loss: 0.0122 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 60/200
53/53 [==============================] - 0s 127us/step - loss: 0.0122 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 61/200
53/53 [==============================] - 0s 102us/step - loss: 0.0121 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 62/200
53/53 [==============================] - 0s 123us/step - loss: 0.0121 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 63/200
53/53 [==============================] - 0s 98us/step - loss: 0.0121 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 64/200
53/53 [==============================] - 0s 108us/step - loss: 0.0121 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 65/200
53/53 [==============================] - 0s 122us/step - loss: 0.0121 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00


Epoch 117/200
53/53 [==============================] - 0s 129us/step - loss: 0.0119 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 118/200
53/53 [==============================] - 0s 108us/step - loss: 0.0119 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 119/200
53/53 [==============================] - 0s 122us/step - loss: 0.0119 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 120/200
53/53 [==============================] - 0s 139us/step - loss: 0.0119 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 121/200
53/53 [==============================] - 0s 140us/step - loss: 0.0119 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 122/200
53/53 [==============================] - 0s 118us/step - loss: 0.0119 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 123/200
53/53 [==============================] - 0s 116us/step - loss: 0.0118 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0

Epoch 174/200
53/53 [==============================] - 0s 108us/step - loss: 0.0116 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 175/200
53/53 [==============================] - 0s 126us/step - loss: 0.0116 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 176/200
53/53 [==============================] - 0s 104us/step - loss: 0.0116 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 177/200
53/53 [==============================] - 0s 124us/step - loss: 0.0116 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 178/200
53/53 [==============================] - 0s 138us/step - loss: 0.0116 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 179/200
53/53 [==============================] - 0s 116us/step - loss: 0.0116 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 180/200
53/53 [==============================] - 0s 126us/step - loss: 0.0116 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0

ShapeletModel(batch_size=256, max_iter=200, n_shapelets_per_size={2: 3},
              optimizer='sgd', random_state=None, shapelet_length=0.15,
              total_lengths=3, verbose=1, verbose_level=None,
              weight_regularizer=0.01)

Ecco le prime shapelet che possiamo plottare (sono un po' piccole, però):

In [117]:
shp_finder.shapelets_

array([array([[7.69612217],
       [7.74837208]]),
       array([[363.04803467],
       [381.07327271]]),
       array([[477.70690918],
       [474.22314453]])], dtype=object)

In [25]:
shp_finder = ShapeletModel(n_shapelets_per_size={5:3},
                        optimizer="sgd",
                        weight_regularizer=.01,
                        max_iter=200,
                        verbose=1)

In [26]:
shp_finder.fit(time_series,classes)

Epoch 1/200
53/53 [==============================] - 1s 24ms/step - loss: 242964.7812 - binary_accuracy: 0.0189 - binary_crossentropy: 242964.7500
Epoch 2/200
53/53 [==============================] - 0s 133us/step - loss: 35516.8750 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 3/200
53/53 [==============================] - 0s 145us/step - loss: 35502.6641 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 4/200
53/53 [==============================] - 0s 120us/step - loss: 35488.4648 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 5/200
53/53 [==============================] - 0s 139us/step - loss: 35474.2734 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 6/200
53/53 [==============================] - 0s 131us/step - loss: 35460.0859 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 7/200
53/53 [==============================] - 0s 117us/step - loss: 35445.9023 - binary_accuracy: 1.0000 - binary_cr

Epoch 57/200
53/53 [==============================] - 0s 130us/step - loss: 34743.9570 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 58/200
53/53 [==============================] - 0s 150us/step - loss: 34730.0586 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 59/200
53/53 [==============================] - 0s 119us/step - loss: 34716.1680 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 60/200
53/53 [==============================] - 0s 118us/step - loss: 34702.2852 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 61/200
53/53 [==============================] - 0s 136us/step - loss: 34688.4023 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 62/200
53/53 [==============================] - 0s 111us/step - loss: 34674.5273 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 63/200
53/53 [==============================] - 0s 118us/step - loss: 34660.6641 - binary_accuracy: 1.0000 - bin

Epoch 113/200
53/53 [==============================] - 0s 119us/step - loss: 33974.2734 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 114/200
53/53 [==============================] - 0s 128us/step - loss: 33960.6797 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 115/200
53/53 [==============================] - 0s 117us/step - loss: 33947.0977 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 116/200
53/53 [==============================] - 0s 132us/step - loss: 33933.5234 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 117/200
53/53 [==============================] - 0s 138us/step - loss: 33919.9453 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 118/200
53/53 [==============================] - 0s 106us/step - loss: 33906.3828 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 119/200
53/53 [==============================] - 0s 151us/step - loss: 33892.8203 - binary_accuracy: 1.000

Epoch 169/200
53/53 [==============================] - 0s 181us/step - loss: 33221.6289 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 170/200
53/53 [==============================] - 0s 103us/step - loss: 33208.3438 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 171/200
53/53 [==============================] - 0s 104us/step - loss: 33195.0625 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 172/200
53/53 [==============================] - 0s 140us/step - loss: 33181.7852 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 173/200
53/53 [==============================] - 0s 138us/step - loss: 33168.5117 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 174/200
53/53 [==============================] - 0s 117us/step - loss: 33155.2461 - binary_accuracy: 1.0000 - binary_crossentropy: 0.0000e+00
Epoch 175/200
53/53 [==============================] - 0s 145us/step - loss: 33141.9883 - binary_accuracy: 1.000

ShapeletModel(batch_size=256, max_iter=200, n_shapelets_per_size={5: 3},
              optimizer='sgd', random_state=None, shapelet_length=0.15,
              total_lengths=3, verbose=1, verbose_level=None,
              weight_regularizer=0.01)

In [27]:
shapelets = shp_finder.shapelets_

In [28]:
shapelets

array([array([[241.93147278],
       [253.50842285],
       [261.47808838],
       [264.63253784],
       [259.35205078]]),
       array([[485.85241699],
       [489.37011719],
       [491.25881958],
       [492.37084961],
       [492.39639282]]),
       array([[6.1582942 ],
       [5.92497683],
       [5.75971317],
       [5.86721373],
       [6.48456383]])], dtype=object)

In pratica, ogni array corrisponde alle coordinate di Light di una shapelet. Per plottarle, bisogna aggiungere un'altra coordinata, che va sulle x e sono le ore: a ogni coordinata di Temperature corrisponde un'ora sull'asse x (non dev'essere un'ora specifica, basta che ci sia scritto "1", per quella successiva "2" etc.